In [1]:
import h5py
import numpy as np
from PIL import Image
import json
import os
import copy

from tomo2mesh.projects.steel_am.coarse2fine import coarse_map, process_subset
from tomo2mesh.misc.voxel_processing import TimerGPU
from tomo2mesh.structures.voids import Voids
from tomo2mesh.porosity.params_3dunet import *
from tomo2mesh.unet3d.surface_segmenter import SurfaceSegmenter
from tomo2mesh.fbp.recon import recon_slice, recon_binned, recon_all

2022-09-23 15:15:34.792010: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-23 15:15:36.754754: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-09-23 15:15:43.368760: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-09-23 15:15:43.369974: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

In [3]:
with open('working_dir/config.json', 'r') as config_f:
    config = json.load(config_f)
prefix = config['img_dir']
start_file = config['img_range'][0]
end_file = config['img_range'][1]
center = config['center']

omega = np.asarray(config['omega'])
omega = omega / 180 * np.pi # NOTE: Why 1pi and not 2pi

In [4]:
projs = []
for im_idx in range(start_file, end_file + 1):
    im = Image.open(os.path.join(prefix, config['img_prefix'] + f'_{im_idx}.tif'))
    projs.append(np.array(im))
projs = np.stack(projs)

In [5]:
b=4
b_K = 4

In [6]:
voids_b = coarse_map(projs, omega, center, b, b_K, 2)

	TIME: COARSE RECON 14.95 secs
	STAT: voids found - 15101
	TIME: LABELING 12.63 secs


In [7]:
filtered_voids = copy.deepcopy(voids_b)
# Insert Filtering Here

In [8]:
filtered_voids.export_void_mesh_mproc("sizes", edge_thresh=0).write_ply(
    f"working_dir/{config['img_prefix']}_{config['img_range'][0]}_{config['img_range'][1]}.ply")

void2mesh_mproc: preserve_feature : False, edge_thres : 0, b : 4, nprocs : None
	TIME: compute void mesh 7.62 secs
